In [1]:
import cv2
import numpy as np
import glob
import os
import pickle

In [3]:
with open("output/20230928_101858/result/val_detections.pkl", 'rb') as f:
    val_detections = pickle.load(f)
with open("output/20230928_101858/result/all_detections_after.pkl", 'rb') as f:
    all_detections = pickle.load(f)

In [7]:
with open("output/20230928_101858/result/val_annotations.pkl", 'rb') as f:
    val_annotations = pickle.load(f)
with open("output/20230928_101858/result/all_annotations_after.pkl", 'rb') as f:
    all_annotations = pickle.load(f)

In [8]:
len(val_detections), len(val_annotations)

(2506, 2506)

In [9]:
len(all_detections), len(all_detections)

(484, 484)

410

In [7]:
data_dir = "/home/agent/Code/yolov3_pytorch/annotation/data_parallel"
img_dir = os.path.join(data_dir, "images/train")
lab_dir = os.path.join(data_dir, "labels/train")
img_paths = glob.glob(f'{img_dir}/*.*')
lab_paths = glob.glob(f'{lab_dir}/*.*')

In [17]:
img = cv2.imread(img_paths[0])

In [18]:
img.shape

(160, 320, 3)

In [12]:
img_paths[0].replace('images', 'labels').replace('png', 'txt').replace('jpg', 'txt')

'/home/agent/Code/yolov3_pytorch/annotation/data_parallel/labels/train/exp10_0.txt'

In [13]:
lab_path = img_paths[0].replace('images', 'labels').replace('png', 'txt').replace('jpg', 'txt')
labels = np.loadtxt(lab_path).reshape(-1, 5)


In [14]:
labels

array([[0.        , 0.60731236, 0.31402428, 0.05173814, 0.03653711]])

In [15]:
filled_labels = np.zeros((10, 5)) # 创建50×5的占位空间
filled_labels[range(len(labels))[:10]] = labels[:10]

In [16]:
filled_labels

array([[0.        , 0.60731236, 0.31402428, 0.05173814, 0.03653711],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [19]:
import torch

In [20]:
H, W = 160, 320
a = torch.arange(H).repeat(W, 1).t().view(1, 1, H, W)

In [21]:
a.size()

torch.Size([1, 1, 160, 320])

In [23]:
torch.arange(H).size(), torch.arange(H).repeat(W, 1).size()

(torch.Size([160]), torch.Size([320, 160]))

In [24]:
nG = (3, 4)
b = torch.ones((1, 2, *nG, 3))

In [25]:
b.size()

torch.Size([1, 2, 3, 4, 3])

In [26]:
anchors = torch.Tensor([[[7,14],[8,16],[10,27]],
                           [[12,21],[16,17],[17,34]],
                           [[17,22],[20,18],[22,20]]])

In [27]:
anchors.size()

torch.Size([3, 3, 2])

In [28]:
anchors[0]

tensor([[ 7., 14.],
        [ 8., 16.],
        [10., 27.]])

In [1]:
from utils import bbox_iou_numpy, bbox_iou
import numpy as np
import torch as th

In [2]:
bbox1 = th.tensor([[0, 0, 2, 2]])
bbox2 = th.tensor([[1, 1, 3, 3], [1, 1, 4, 4]])

In [3]:
output = bbox_iou(bbox1, bbox2)

In [9]:
def calculate_iou(rect1, rectangles):
    # 计算交集区域的左上角坐标
    x_intersection = np.maximum(rect1[0], rectangles[:, 0])
    y_intersection = np.maximum(rect1[1], rectangles[:, 1])
    
    # 计算交集区域的右下角坐标
    x_intersection_end = np.minimum(rect1[2], rectangles[:, 2])
    y_intersection_end = np.minimum(rect1[3], rectangles[:, 3])
    
    # 计算交集区域的宽度和高度（可能为负数，表示没有重叠）
    intersection_width = np.maximum(0, x_intersection_end - x_intersection)
    intersection_height = np.maximum(0, y_intersection_end - y_intersection)
    
    # 计算交集区域的面积
    intersection_area = intersection_width * intersection_height
    
    # 计算矩形1的面积
    area_rect1 = (rect1[2] - rect1[0]) * (rect1[3] - rect1[1])
    
    # 计算其他矩形的面积
    area_rectangles = (rectangles[:, 2] - rectangles[:, 0]) * (rectangles[:, 3] - rectangles[:, 1])
    
    # 计算并集区域的面积
    union_area = area_rect1 + area_rectangles - intersection_area
    
    # 计算IoU
    iou = intersection_area / union_area
    
    return iou

# 示例矩形坐标
rect1 = np.array([0, 0, 4, 4])
rectangles = np.array([(2, 2, 6, 6), (5, 5, 8, 8), (10, 10, 12, 12)])

# 计算矩形1与多个矩形的IoU
iou_values = calculate_iou(rect1, rectangles)
iou_values


array([0.14285714, 0.        , 0.        ])

In [1]:
import pickle

In [3]:
with open('pred_res.pkl', 'rb') as f:
    pred_res = pickle.load(f)
with open('target_res.pkl', 'rb') as f:
    target_res = pickle.load(f)


In [9]:
target_res[-1][0]

tensor([[0.0000, 0.4851, 0.2456, 0.0259, 0.0494],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]], device='cuda:0',
       dtype=torch.float64)

In [18]:
pred_res[-1][0][pred_res[-1][0][:, 4] > 0.5]

tensor([[149.1717,  38.5623, 158.9098,  51.4984,   0.9999,   0.7769],
        [151.4039,  34.3758, 162.4739,  45.4148,   0.9998,   0.5860],
        [151.3564,  32.7130, 163.5119,  43.7672,   0.9994,   0.7541],
        [155.2812,  32.0590, 166.1803,  42.3762,   0.9096,   0.6170],
        [149.8114,  35.4669, 160.4946,  46.9297,   0.9793,   0.6962],
        [147.6945, 115.2550, 164.9058, 129.6601,   0.8271,   0.6869]],
       device='cuda:0')

In [1]:
from dataset import ListDataset
import torch
# 循环读取图片，用imshow展示

# ListDataset是用于训练时使用的数据集类, 它会返回以下三个变量:
# 图片路径(str), 图片(3,416,416), 以及图片的box标签信息(50,5)
dataloader = torch.utils.data.DataLoader(
    ListDataset("annotation/data_parallel", mode='test'), batch_size=4
)
# anchors = torch.Tensor([[[7,14],[8,16],[10,27]],
#                         [[12,21],[16,17],[17,34]],
#                         [[17,22],[20,18],[22,20]]])
anchors = torch.Tensor([[[10, 13], [16, 30], [33, 23]],
                        [[30, 61], [62, 45], [59, 119]],
                        [[116, 90], [156, 198], [373, 326]]])

all_detections = []
all_annotations = []
img_path, target_res = [], []
for batch_i, (img_path, imgs, targets) in enumerate(dataloader):
    print(img_path)
    print(imgs.shape)
    print(targets.shape)
    break


('annotation/data_parallel/images/test/exp15_187.png', 'annotation/data_parallel/images/test/exp5_282.png', 'annotation/data_parallel/images/test/exp5_134.png', 'annotation/data_parallel/images/test/exp5_128.png')
torch.Size([4, 3, 160, 320])
torch.Size([4, 10, 5])
